In [3]:
using Pkg
Pkg.activate(".")

using SumOfSquares
using DynamicPolynomials
using MosekTools
using LinearAlgebra

# Parameters 
ϵ = 10^(-5)
λ = -1
sostol = 4

@polyvar z

# Using Mosek as the SDP solver
solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)

function bc_suf(deg, num_tech)
    # synthesize BC by using the sufficient condition
    # deg: degree of BC template
    # num_tech: whether to use intermediate enhancement techniques (=1) or not (=0)

    # compute init and unsafe region
    mi = length(gi)
    mu = length(gu)
    init = @set(gi[1]>=0)
    for i = 2:mi
        init = intersect(init, @set(gi[i]>=0))
    end
    unsafe = @set(gu[1]>=0)
    for i = 2:mu
        unsafe = intersect(unsafe, @set(gu[i]>=0))
    end
        
    model = SOSModel(solver)
    monos = monomials(vars, 0:deg)
    
    if tech == 1
        # use scaledmonomial basis
        monos = ScaledMonomialBasis(monos)
    end
    
    @variable(model, B, Poly(monos))
    @variable(model, γ)
    dBdt = dot(differentiate(B, vars), f)
    
    @constraint(model, - B - γ >= 0, domain = init, maxdegree =  maxdegree(B)+sostol)
    @constraint(model, B - γ - ϵ >= 0, domain = unsafe, maxdegree =  maxdegree(B)+sostol)
    @constraint(model, λ*B - dBdt - γ >= 0, maxdegree =  maxdegree(dBdt)+sostol)
    @objective(model,Max,γ)
    
    JuMP.optimize!(model)
    if (JuMP.has_values(model)&&SumOfSquares.value(γ)>= 0)
        B_val = SumOfSquares.value(B)
        if tech==1
            coef_list = coefficients(B_val);
            for i in 1:length(coef_list)
                if coef_list[i] <= ϵ && coef_list[i] >= -ϵ
                    coef_list[i] = 0
                end
            end
            B_val = dot(coef_list, monomials(B_val))
        end
        return B_val
    else
        return 0
    end
end

function homo(f)
    # homogenize f w.r.t. variable z (denoted x_0 in paper)
    
    f_homo = 0
    d = maxdegree(f)
    for t in terms(f)
        f_homo += t*z^(d-degree(t))
    end
    return f_homo
end

function bc_nec(deg, tech)
    # synthesize BC by using the necessary condition (homogenization formulation)
    # deg: degree of BC template
    # num_tech: whether to use intermediate enhancement techniques (=1) or not (=0)

    mi = length(gi)
    mu = length(gu)
    init = @set(gi[1]>=0)
    for i = 2:mi
        init = intersect(init, @set(gi[i]>=0))
    end
    unsafe = @set(gu[1]>=0)
    for i = 2:mu
        unsafe = intersect(unsafe, @set(gu[i]>=0))
    end
    
    init_h = @set(homo(gi[1])>=0)
    for i = 2:length(gi)
        init_h = intersect(init_h, @set(homo(gi[i])>=0))
    end
    unsafe_h = @set(homo(gu[1])>=0)
    for i = 2:length(gu)
        unsafe_h = intersect(unsafe_h, @set(homo(gu[i])>=0))
    end

    θ = z^2
    for i = 1:length(vars)
        θ = θ + vars[i]^2
    end

    init_h = intersect(init_h, @set(θ==1));
    unsafe_h = intersect(unsafe_h, @set(θ==1));

    model = SOSModel(solver)
    monos = monomials(vars, 0:deg)
    
    if tech == 1
        monos = ScaledMonomialBasis(monos)    
    end
        
    @variable(model, B, Poly(monos))
    @variable(model, γ)
    dBdt = dot(differentiate(B, vars), f)

    @constraint(model, - homo(B+γ) + ϵ >= 0 , domain = intersect(init_h,@set(z>=0)), maxdegree = maxdegree(B)+sostol)
    @constraint(model,   homo(B-γ) + ϵ >= 0 , domain = intersect(unsafe_h,@set(z>=0)), maxdegree =  maxdegree(B)+sostol)    
    @constraint(model,   homo(λ*B - dBdt - γ) + ϵ >= 0, domain = intersect(@set(θ==1),@set(z>=0)), maxdegree =  maxdegree(dBdt)+sostol)        
    @objective(model, Max, γ)
        
    JuMP.optimize!(model)
    if (JuMP.has_values(model)&&SumOfSquares.value(γ)>=0)
        #println("A feasible solution is found! Optimal Value: ",SumOfSquares.value(γ))
        B_val_h = SumOfSquares.value(B)
        if tech == 1
            coef_list = coefficients(B_val_h);
            for i in 1:length(coef_list)
                if coef_list[i] <= ϵ && coef_list[i] >= -ϵ
                    coef_list[i] = 0
                end
            end
            B_val_h = dot(coef_list, monomials(B_val_h))
        end
        return B_val_h
    else 
        return 0
    end
end


  Activating project at `~/Documents/code/BCunbounded`


bc_nec (generic function with 1 method)

In [31]:
function bcp(degB, degH, num_tech)
    # synthesize BC by using the sufficient condition
    # deg: degree of BC template
    # num_tech: whether to use intermediate enhancement techniques (=1) or not (=0)

    # compute init and unsafe region
    mi = length(gi)
    mu = length(gu)
    md = length(gd)
    init = @set(gi[1]>=0)
    for i = 2:mi
        init = intersect(init, @set(gi[i]>=0))
    end
    unsafe = @set(gu[1]>=0)
    for i = 2:mu
        unsafe = intersect(unsafe, @set(gu[i]>=0))
    end
    range = @set(gd[1]>=0)
    for i = 2:md
        range = intersect(range, @set(gd[i]>=0))
    end
        
    model = SOSModel(solver)
    monos = monomials(vars, 0:degB)
    hmonos = monomials(hvars, 0:degH)
    
    if tech == 1
        # use scaledmonomial basis
        monos = ScaledMonomialBasis(monos)
    end

    
    @variable(model, B, Poly(monos))
    @variable(model, H, Poly(hmonos))
    
    dBdt = dot(differentiate(B, vars), f)
    
    @constraint(model, - B >= 0, domain = init, maxdegree =  maxdegree(B)+sostol)
    @constraint(model, B - ϵ >= 0, domain = unsafe, maxdegree =  maxdegree(B)+sostol)
    @constraint(model, H - (dBdt - L*B) >= 0, domain = range, maxdegree =  maxdegree(H)+sostol)
    
    moments = [2, 0, 2/3, 0, 2/5, 0, 2/7, 0, 2/9, 0, 2/11]
    target = dot(coefficients(H), moments[1:degH+1])

    @objective(model, Min, target)
    
    JuMP.optimize!(model)
    if (JuMP.has_values(model))
        B_val = SumOfSquares.value(B)
        if tech==1
            coef_list = coefficients(B_val);
            for i in 1:length(coef_list)
                if coef_list[i] <= ϵ && coef_list[i] >= -ϵ
                    coef_list[i] = 0
                end
            end
            B_val = dot(coef_list, monomials(B_val))
        end
        H_val = SumOfSquares.value(H)
        
        return (H_val, B_val)

    else
        return (0, 0) 
    end
end

            
# number of variables
@polyvar x1 x2
@polyvar y

vars = [x1, x2]
hvars = [y]

L = y;
# vector field
f = [ x2,
     -x1 + (1/3)*x1^3 - x2]

# description polynomial of the initial/unsafe set, >=0 by default
gi = [ 0.25 - (x1-1.5)^2 - (x2)^2 ]  
gu = [ 0.16 - (x1+1)^2 - (x2+1)^2 ]
gd = [ 4-x, x+4, 1-y, y-1]

(H, B) = bcp(2, 2, tech)

(-2.525050175023774 + 12.992259236821557y + 6.806018007812784y², 2.1749332584555545 - 0.8889550081563484x2 - 3.2852777571840894x1 + 0.0002630063455425415x2² - 2.4380442456732334x1x2 - 1.0594636564523519x1²)

In [25]:
@polyvar x

coefficients(3*x+2*x^2+1)


moments = [2, 0, 2/3, 0, 2/5, 0, 2/7, 0, 2/9, 0, 2/11]
moments[1:3]


3-element Vector{Float64}:
 2.0
 0.0
 0.6666666666666666

In [4]:
# search BC for the given benchmark system

degMin = 1
degMax = 6
tech = 1
name = "arch2-1"

include("./Benchmarks/"*name*".jl");

# print system
file = open("./Results/systems/"*name*".txt", "w");
for k = [vars,f,gi,gu]
    write(file, "{")
    for i = 1:length(k)-1
        write(file, string(k[i])*",")
    end
    write(file, string(last(k))*"}\n")
end
close(file)

# print sufficient condition results
file = open("./Results/sufficient/"*name*".txt", "w");
for deg = degMin:degMax
    stats = @timed B = bc_suf(deg,tech)
    write(file, Base.replace(string(B),"e"=>"*10^")*"\n")
    write(file, string(stats.time)*"\n") 
end
close(file)

# print homogenization approach results
file = open("./Results/necessary/"*name*".txt", "w");
for deg = degMin:degMax
    stats = @timed B = bc_nec(deg,tech)
    write(file, Base.replace(string(B),"e"=>"*10^")*"\n")
    write(file, string(stats.time)*"\n") 
end
close(file)


In [5]:
# run all benchmarks with specified BC degrees

tech = 0

benchmarks = ["vector-1", "vector-2", "barrier-1", "barrier-2", "lie-der-1", "lie-der-2", 
    "arch1-1", "arch1-2", "arch2-1", "arch2-2", "arch3-1", "arch3-2", "arch4-1", "arch4-2",
    "nagumo-1", "nagumo-2", "lotka-1", "lotka-2", "lorenz-1", "lorenz-2", "lyapunov-1", "lyapunov-2"]

for name in benchmarks
    include("./Benchmarks/"*name*".jl");
    
    # record system information (used for verification)
    file = open("./Results/systems/"*name*".txt", "w");
    for k = [vars,f,gi,gu]
        write(file, "{")
        for i = 1:length(k)-1
            write(file, string(k[i])*",")
        end
        write(file, string(last(k))*"}\n")
    end
    close(file)

    println(name)
    # sufficient condition
    file = open("./Results/sufficient/"*name*".txt", "w");
    stats = @timed B = bc_suf(bc_deg,tech)
    write(file, Base.replace(string(B),"e"=>"*10^")*"\n")
    write(file, string(stats.time)*"\n")
    println("suf: ", stats.time)
    close(file)
   
    # necessary condition
    file = open("./Results/necessary/"*name*".txt", "w");
    stats = @timed B = bc_nec(bc_deg,tech)
    write(file, Base.replace(string(B),"e"=>"*10^")*"\n")
    write(file, string(stats.time)*"\n")
    println("nec: ", stats.time)
    close(file)
end



vector-1
suf: 0.727090125
nec: 0.094575
vector-2
suf: 0.205319875
nec: 0.044607417
barrier-1
suf: 0.164291625
nec: 0.169610916
barrier-2
suf: 0.008200916
nec: 0.125984166
lie-der-1
suf: 0.291658709
nec: 0.021145917
lie-der-2
suf: 0.07102875
nec: 0.229182584
arch1-1
suf: 0.049024791
nec: 0.457486416
arch1-2
suf: 0.022007833
nec: 0.08650825
arch2-1
suf: 0.014616458
nec: 0.066097
arch2-2
suf: 0.007157584
nec: 0.0212235
arch3-1
suf: 0.008767833
nec: 0.027136917
arch3-2
suf: 0.006873292
nec: 0.030468709
arch4-1
suf: 0.010183709


LoadError: InterruptException:

In [1]:
using Pkg
Pkg.activate(".")

using SumOfSquares
using DynamicPolynomials
using MosekTools
using LinearAlgebra

@polyvar x1 x2

M = x1^2+x2^2
#x1^2*x2^4 + x1^4*x2^2 - 3*x1^2*x2^2 +1

solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)
model = SOSModel(solver)

@constraint(model, M>=0)
JuMP.optimize!(model)

  Activating project at `~/Documents/code/BCunbounded`


In [2]:
JuMP.has_values(model)

false

In [10]:
@polyvar x0 x1 x2

M_h = x1^2*x2^4 + x1^4*x2^2 - 3*x1^2*x2^2*x0^2 + x0^6

X = monomials([x0, x1, x2], )

solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)
model2 = SOSModel(solver)

@constraint(model2, M_h>=0, domain = intersect(@set(x0>=0), @set(x0^2+x1^2+x2^2-1==0)))
JuMP.optimize!(model2)

In [11]:
JuMP.has_values(model2)

false